In [ ]:
import pickle
import pandas as pd
import numpy as np
import scipy
import joblib

In [ ]:
# Load ensemble of models

models_dict = joblib.load('models_dict.joblib')
models_dict.keys()

In [ ]:
# Load data: Data should be formatted as sample cohort. See README for example

df_cohort = pd.read_csv('sample_cohort.csv')
test_hosp, test_window, test_y = df_cohort['hosp_id'], df_cohort['window_id'], df_cohort['y']
cohort_IDs = df_cohort.set_index('ID')[[]]

In [ ]:
df_cohort

In [ ]:
# Functions to assist with evaluation 

from sklearn import metrics, utils
from joblib import Parallel, delayed


def bootstrap_func(i, y_true, y_score):
    # Bootstrap resample to calculate AUROC
    yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
    return metrics.roc_curve(yte_true_b, yte_pred_b), metrics.roc_auc_score(yte_true_b, yte_pred_b)

def get_roc_CI(y_true, y_score):
    # Bootstrap confidence intervals 
    roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func)(i, y_true, y_score) for i in range(1000)))
    print('Test AUC: ({:.3f}, {:.3f}) percentile 95% CI'.format(np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    for fpr, tpr, _ in roc_curves:
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        aucs.append(metrics.auc(fpr, tpr))

    mean_tpr = np.mean(tprs, axis=0)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + 1.96 * std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - 1.96 * std_tpr, 0)
    return roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper

def eval3():
    # Calculate hospital admission level AUROC for every complete window
    df_Yte = df_Yte_all.copy()
    df_Yte = df_Yte[df_Yte['window_id'] >= 1]
    df_Yte_agg = df_Yte.groupby(['hosp_id']).max()
    return df_Yte_agg

## M-CURES model performance and scores

In [ ]:
# Load exact model and features of patients. Features should be generated from preprocessing script. 

mcures_clfs = models_dict['M-CURES']
df_mcures = pd.read_csv('../preprocessing/sample_output/mcures.csv').set_index('ID')

In [ ]:
df_mcures

In [ ]:
### AUROC on given dataset

# Calculate model outputs for all patients, average over all models
eval_matrix = scipy.sparse.csr_matrix(cohort_IDs.join(df_mcures).values.astype(float))
all_y = np.array([clf.predict_proba(eval_matrix)[:,1] for clf in mcures_clfs])
y_scores = all_y.mean(0)

# To evaluate models, take maximum over all windows
df_Yte_all = pd.DataFrame({'hosp_id': test_hosp, 'window_id': test_window, 'y': test_y, 'y_score': y_scores})
df_Yte_agg = eval3()
y_score = df_Yte_agg['y_score']
y_true = df_Yte_agg['y']
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
print('Test AUC: {:.3f}'.format(metrics.roc_auc_score(y_true, y_score)))

# # Optionally: Generate 95% CI
# try:
#     roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper = get_roc_CI(y_true, y_score)
# except:
#     pass

In [ ]:
# Generate list of scores for each example 

y_score_lst = df_Yte_all.groupby(['hosp_id'])['y_score'].apply(list)
df1 = pd.DataFrame({'y_scores_mcures_lst':  y_score_lst})
df2 = pd.DataFrame({'id': df_Yte_agg.index, 'y_scores_mcures': y_score})

outcome_outputs = df1.merge(df2, left_index=True, right_index=True)

In [ ]:
outcome_outputs 